<a href="https://colab.research.google.com/github/ArghyaPal/Adversarial_Data_Programming/blob/master/Adversarial_Data_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial Data Programming
Paucity of large curated hand-labeled training data forms a major bottleneck in the deployment of machine learning models in computer vision and other fields. Recent work (Data Programming) has shown how distant supervision signals in the form of labeling functions can be used to obtain labels for given data in near-constant time. In this work, we present Adversarial Data Programming (ADP), which presents an adversarial methodology to generate data as well as a curated aggregated label, given a set of weak labeling functions. We validated our method on the MNIST, Fashion MNIST, CIFAR10 and SVHN datasets, and it outperformed many state-of-the-art models. We conducted extensive experiments to study its usefulness, as well as showed how the proposed ADP framework can be used for transfer learning as well as multi-task learning, where data from two domains are generated simultaneously using the framework along with the label information. Our future work will involve understanding the theoretical implications of this new framework from a game-theoretic perspective, as well as explore the performance of the method on more complex datasets.

**TL;DR:**<br>
Learning the parameters of Joint Distribution, i.e. $P(Χ, y)$ and $Χ$: image, $y$: label, using Distant Supervision Signals. In this work, we seek to learn the parameters of the Joint Distribution $P(Χ, LFB_{(L, θ, φ)}(X))$. The $LFB_{(L, θ, φ)} (.)$ acts as a proxy to the real label y. 

In [ ]:
# Please run it once on a runtime
!pip install kmeans-pytorch

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np
from kmeans_pytorch import kmeans, kmeans_predict

In [ ]:
# Decide which device we want to run on
ngpu = 4
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
'''
Hyperparameters used in this projects
'''
# The noise dimension for the Generator, i.e. the z dimension, sampled from a Gaussian Distribution
latent_size = 64
# The network layer dimension of Generator and Discriminator
hidden_size = 256
# The generated image size/image size. MNIST size 28 X 28 = 784
image_size = 784
# Number of epochs
num_epochs = 300
# A mini-batch size
batch_size = 20
# Adam learning rate
lr = 0.0001

### Please mention the number of Labeleing Functions here

In [ ]:
theta = 1

### Dataset

In [ ]:
sample_dir = 'samples'
save_dir = 'save'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5),   # 3 for RGB channels
                                     std=(0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

### ADP Labeling Function Block $LFB(y|X, \Theta)$

In [ ]:
    """
    The labeling functions infer label of an image independently 
    (i.e. independent decision assumption) and the parameter gives relative 
    weight to each of the labeling functions based on their correctness of 
    inferred label for true class y.

    Parameters
    ----------
    X
        Batch of images, torch.Tensor
    theta
        Batch of relative accuracies are weights given to labeling functions 
        based on whether their outputs agree with true class label y of an 
        image X, torch.Tensor
    Returns
    -------
    theta * y1
        Batch of approximated labels (in this formulation), torch.Tensor
    """
def lf1(fake_images, theta): 
  # fake_images is a batch of generated images from the G_image
  cluster_centers = torch.load('/content/drive/My Drive/Colab_Notebooks/cluster_centers.pt')
  # we use k-means (unsupervised) clustering as one labeling function
  y1 = kmeans_predict(fake_images, cluster_centers, 'euclidean', device=device)
  y1 = y1.view(-1,1).cuda()
  return theta * y1

### ADP Generator $G(X, \Theta|z)$

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.G_common = nn.Sequential(
                              nn.Linear(latent_size, hidden_size),
                              nn.ReLU(),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU())
        self.G_image = nn.Sequential(
                              nn.Linear(hidden_size, image_size),
                              nn.Tanh())
        self.G_parameter = nn.Sequential(
                              nn.Linear(hidden_size, theta),
                              # We use softmax to make the theta to be in the 
                              # range [0,1]
                              nn.Softmax(dim=1))
    """
    Parameters
    ----------
    z
        Batch of noise vector, torch.Tensor
    Returns
    -------
    X
        Batch of images, torch.Tensor
    theta
        Batch of a Relative accuracies, torch.Tensor
    """
    def forward(self, input):
        common = self.G_common(input)
        image = self.G_image(common)
        theta = self.G_parameter(common)
        return image, theta

### ADP Discriminator D(X, y)


In [ ]:
class Discriminator(nn.Module):
  '''
  The discriminator class takes a batch of either real or generated image and 
  inferred label from Labeling Functions Block LFB(.) pairs as input.
  And, maps that to a probability score to estimate the aforementioned 
  likelihood of the image-label pair as real labeled-image
  '''
    def __init__(self):
        super(Discriminator, self).__init__()
        '''
        The D_image takes a batch of images
        '''
        self.D_image = nn.Sequential(
                              nn.Linear(image_size, hidden_size),
                              nn.LeakyReLU(0.2))
        '''
        The D_label takes a batch of labels
        '''
        self.D_label = nn.Sequential(
                              nn.Linear(10, hidden_size),
                              nn.LeakyReLU(0.2))
        '''
        The D_common adds the D_image and D_label branches and provides 
        '''
        self.D_common = nn.Sequential(
                              nn.Linear(hidden_size*2, hidden_size),
                              nn.LeakyReLU(0.2),
                              nn.Linear(hidden_size, 1),
                              # This will ensure the output to be in the range of [0,1]
                              nn.Sigmoid())
    """Estimates the probability score as a likelihood of the input image-label pair
    as real image-label pair.
    This function provides a batch of probability scores, i.e. P(X, y), stored as a 
    vector P. We will use that vector to give loss signal to the Discriminator D(.) 
    and Generator G(.).

    Parameters
    ----------
    X
        Batch of images, torch.Tensor
    y
        Batch of labels, torch.Tensor
    Returns
    -------
    result
        Batch of probabilities, torch.Tensor
    """
    def forward(self, image, label):
        # image is the batch of images given to the D_image branch
        image = self.D_image(image)
        # label is the corresponding labels given to the D_label branch
        label = self.D_label(label)
        # common is the branch that concates D_image and D_label
        common = torch.cat([image, label], 1)
        # result is the batch of probability score
        result = self.D_common(common)
        return result

In [ ]:
# Device setting
G = Generator().cuda()
D = Discriminator().cuda()

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
# Statistics to be saved
d_losses = np.zeros(num_epochs)
g_losses = np.zeros(num_epochs)
real_scores = np.zeros(num_epochs)
fake_scores = np.zeros(num_epochs)

In [ ]:
# We will start Adversarial Data Programming training
total_step = len(data_loader)

# To change num_epochhs please visit the hyperparameters section
for epoch in range(num_epochs):
  # We will be training discriminator one time and then the generator one time
  # using the for loop
    for i, (images, original_labels) in enumerate(data_loader):

        # Reading real MNIST images
        images = images.view(batch_size, -1).cuda()
        images = Variable(images)
        # Reading real MNIST labeles of the corresponding images
        original_labels = torch.nn.functional.one_hot(original_labels, num_classes=10)
        # We are making it in one-hot encoding
        original_labels = original_labels.view(batch_size, -1).cuda()
        original_labels = Variable(original_labels).float()
        

        # Labels for the discriminator. Real Labels are 1 and Fake Labels are 0
        real_labels = torch.ones(batch_size, 1).cuda()
        real_labels = Variable(real_labels)
        fake_labels = torch.zeros(batch_size, 1).cuda()
        fake_labels = Variable(fake_labels)

        #######################################################################
        '''
        Below line of codes will train the discriminator.
        The discriminator will take: (i) a batch of images; and (ii) a batch of labels
        The object, i.e. outputs = D(images, original_labels), of the Discriminator class
        We compute BCE_Loss using real images where:
        BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        Second term of the loss is always zero since real_labels == 1
        '''
        outputs = D(images, original_labels)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        #######################################################################

        #######################################################################
        '''
        We sample z from the Isotropic Gaussian distribution
        Compute BCELoss using fake images
        First term of the loss is always zero since fake_labels == 0
        '''
        z = torch.randn(batch_size, latent_size).cuda()
        z = Variable(z)
        # We put that in the G_common
        fake_images, theta = G(z)

        # generated labels from labeling function (here, we are showing one labeling function)
        gen_labels = lf1(fake_images, theta)
        gen_labels = gen_labels.view(20)
        gen_labels = torch.nn.functional.one_hot(gen_labels.to(torch.int64), num_classes=10).float()
        # 
        outputs = D(fake_images, gen_labels)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        # If D is trained so well, then don't update
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        #######################################################################

        '''
        Train generator 
        '''

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).cuda()
        z = Variable(z)
        fake_images, theta = G(z)
        gen_labels = lf1(fake_images, theta)
        gen_labels = gen_labels.view(batch_size)
        gen_labels = torch.nn.functional.one_hot(gen_labels.to(torch.int64), num_classes=10).float()
        # 
        outputs = D(fake_images, gen_labels)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        # if G is trained so well, then don't update
        reset_grad()
        g_loss.backward()
        g_optimizer.step()


         #=================================================================== #
        #                          Update Statistics                          #
        # =================================================================== #
        d_losses[epoch] = d_losses[epoch]*(i/(i+1.)) + d_loss.data*(1./(i+1.))
        g_losses[epoch] = g_losses[epoch]*(i/(i+1.)) + g_loss.data*(1./(i+1.))
        real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().data*(1./(i+1.))
        fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().data*(1./(i+1.))


        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.data, g_loss.data, 
                          real_score.mean().data, fake_score.mean().data))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    print(gen_labels)
    save_image(denorm(fake_images.data), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
    # Save and plot Statistics
    np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
    np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
    np.save(os.path.join(save_dir, 'fake_scores.npy'), fake_scores)
    np.save(os.path.join(save_dir, 'real_scores.npy'), real_scores)
    
    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    plt.plot(range(1, num_epochs + 1), d_losses, label='d loss')
    plt.plot(range(1, num_epochs + 1), g_losses, label='g loss')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'loss.pdf'))
    plt.close()

    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    pylab.ylim(0, 1)
    plt.plot(range(1, num_epochs + 1), fake_scores, label='fake score')
    plt.plot(range(1, num_epochs + 1), real_scores, label='real score')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'accuracy.pdf'))
    plt.close()

    # Save model at checkpoints
    if (epoch+1) % 50 == 0:
        torch.save(G.state_dict(), os.path.join(save_dir, 'G--{}.ckpt'.format(epoch+1)))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D--{}.ckpt'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/300], Step [200/3000], d_loss: 0.3913, g_loss: 4.5863, D(x): 0.97, D(G(z)): 0.24
Epoch [0/300], Step [400/3000], d_loss: 0.0727, g_loss: 4.7496, D(x): 0.97, D(G(z)): 0.02
Epoch [0/300], Step [600/3000], d_loss: 0.1569, g_loss: 5.6661, D(x): 0.95, D(G(z)): 0.02
Epoch [0/300], Step [800/3000], d_loss: 0.2424, g_loss: 6.4872, D(x): 0.91, D(G(z)): 0.04
Epoch [0/300], Step [1000/3000], d_loss: 0.2634, g_loss: 4.3639, D(x): 0.99, D(G(z)): 0.07
Epoch [0/300], Step [1200/3000], d_loss: 0.1355, g_loss: 3.2382, D(x): 0.96, D(G(z)): 0.03
Epoch [0/300], Step [1400/3000], d_loss: 0.0972, g_loss: 5.6429, D(x): 0.96, D(G(z)): 0.01
Epoch [0/300], Step [1600/3000], d_loss: 0.2605, g_loss: 3.4289, D(x): 0.94, D(G(z)): 0.11
Epoch [0/300], Step [1800/3000], d_loss: 0.0660, g_loss: 4.2444, D(x): 0.97, D(G(z)): 0.03
Epoch [0/300], Step [2000/3000], d_loss: 0.0392, g_loss: 4.2151, D(x): 1.00, D(G(z)): 0.04
Epoch [0/300], Step [2200/3000], d_loss: 0.1175, g_loss: 3.7520, D(x): 1.00, D(G(z)): 0.08
Epo

KeyboardInterrupt: ignored